In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Ichimoku Breakout — Daily Scheduler with Entries, Exits, Telegram, and Auto-Append Portfolio
--------------------------------------------------------------------------------------------

RUN AFTER MARKET CLOSE (EOD).

What it does
------------
1) Scans your universe and builds NEXT-DAY entry plan from Ichimoku breakout:
     - Base: Close crosses ABOVE CloudTop AND SpanA > SpanB
     - Optional confirmations: Tenkan>Kijun, Close>Kijun, Chikou proxy, min breakout %, min cloud thickness, Volume surge.
2) Filters: within X% of 52w high; optional liquidity checks.
3) Ranks by VOLAR vs benchmark + breakout distance; selects top-K entries.
4) **Auto-append entries**: By default, assumes you will take the signals and appends them to
   `portfolio_positions.csv` as `status=open` with:
     - `entry_date` = next weekday after `as_of` (no holiday calendar),
     - `entry_price` = estimated (today close proxy),
     - `shares` = estimated via deployable cash cap,
     - `stop_price` / `target_price` derived from Config.
   (Toggle via `CFG.auto_append_entries`.)
5) Generates EXIT plan for existing open trades (indicator exit or SL/TP hit on EOD).
6) Writes artifacts under outputs/YYYY-MM-DD/:
     - next_day_entries.{csv,json}
     - next_day_exits.{csv,json}
7) Optional Telegram alerts for entries/exits.

Portfolio file
--------------
- Path: CFG.portfolio_path (default: portfolio_positions.csv)
- Auto-created if missing, with headers.
- Minimal columns: ticker, entry_date, entry_price, shares, status
- Optional: stop_price, target_price, exit_date, exit_price, notes
- De-dup: won’t add a new row if an OPEN row for the same ticker already exists.

Notes
-----
- Entry/exit executions are *not* automated; this is a planner/tracker.
- Entry prices/qty are estimates (using EOD close as next-open proxy).
- Exit checks are EOD-based (no intraday).
- Next session date helper skips weekends only (not official holidays).

"""

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception:
    yf = None

try:
    import requests
except Exception:
    requests = None

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("ichimoku_scheduler_v3")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # --- Data / I/O ---
    start_date: str = "2015-01-01"
    end_date: Optional[str] = None
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_root: str = "outputs"
    portfolio_path: str = "portfolio_positions.csv"

    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")

    # --- Ichimoku (classic) ---
    tenkan_len: int = 9
    kijun_len:  int = 26
    spanb_len:  int = 52

    # --- Confirmation toggles ---
    use_ichimoku_confirms: bool = True
    confirm_tenkan_gt_kijun: bool = True
    confirm_close_gt_kijun: bool = True
    confirm_chikou_above: bool = True
    confirm_min_break_pct: float = 0.005
    confirm_min_cloud_thickness_pct: float = 0.01

    use_volume_confirm: bool = True
    volume_surge_mult: float = 1.5
    volume_ma_len: int = 20

    # --- Candidate filters ---
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.70
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

    # --- Ranking & selection ---
    volar_lookback: int = 252
    top_k_daily: int = 5

    # --- “Paper” cash model for entry sizing estimates ---
    current_cash_inr: float = 500_000.0
    deploy_cash_frac: float = 0.25
    per_pick_min_inr: float = 5_000.0

    # --- Stops/Targets (used for both entry plan estimates and exit checks) ---
    stop_loss_pct: float = 0.05
    target_pct: float    = 0.10

    # --- Telegram (optional) ---
    telegram_enabled: bool = False
    telegram_bot_token: str = ""
    telegram_chat_id: str = ""
    alert_on_entries: bool = True
    alert_on_exits: bool = True

    # --- Auto-append entries into portfolio ---
    auto_append_entries: bool = True      # << default ON per your request

CFG = Config()

# =========================
# Helpers / I/O
# =========================
def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def out_dir_for_today(root: str) -> str:
    d = today_str()
    path = os.path.join(root, d)
    ensure_dir(path)
    return path

def next_weekday(d: pd.Timestamp) -> pd.Timestamp:
    # simple next business day: skips Sat/Sun only (no holiday calendar)
    x = d + pd.Timedelta(days=1)
    while x.weekday() >= 5:  # 5=Sat, 6=Sun
        x += pd.Timedelta(days=1)
    return x

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS") and not s.startswith("^"):
            s = f"{s}.NS"
        out.append(s)
    uniq = []
    seen = set()
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dir(cache_dir)
    data: Dict[str, pd.DataFrame] = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^','_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    continue
            except Exception:
                pass
        try:
            if yf is None:
                raise RuntimeError("yfinance not available in this environment")
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)
            df = df[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

# =========================
# Indicators & Signals
# =========================
def rolling_mid(high: pd.Series, low: pd.Series, length: int) -> pd.Series:
    hh = high.rolling(length).max()
    ll = low.rolling(length).min()
    return (hh + ll) / 2.0

def compute_ichimoku_base(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["tenkan"] = rolling_mid(out["High"], out["Low"], cfg.tenkan_len)
    out["kijun"]  = rolling_mid(out["High"], out["Low"], cfg.kijun_len)
    out["span_a_base"] = (out["tenkan"] + out["kijun"]) / 2.0
    out["span_b_base"] = rolling_mid(out["High"], out["Low"], cfg.spanb_len)
    out["cloud_top"] = out[["span_a_base", "span_b_base"]].max(axis=1)
    out["cloud_bot"] = out[["span_a_base", "span_b_base"]].min(axis=1)

    out["cloud_thickness"]  = (out["cloud_top"] - out["cloud_bot"]) / out["Close"]
    out["dist_above_cloud"] = (out["Close"] / out["cloud_top"]) - 1.0

    out["avg_vol_ma"] = out["Volume"].rolling(cfg.volume_ma_len).mean()
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"]   = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def entry_signal_series(d: pd.DataFrame, cfg: Config) -> pd.Series:
    cross_up  = (d["Close"].shift(1) <= d["cloud_top"].shift(1)) & (d["Close"] > d["cloud_top"])
    span_bull = d["span_a_base"] > d["span_b_base"]
    sig = cross_up & span_bull

    if cfg.use_ichimoku_confirms:
        confs = []
        if cfg.confirm_tenkan_gt_kijun:
            confs.append(d["tenkan"] > d["kijun"])
        if cfg.confirm_close_gt_kijun:
            confs.append(d["Close"] > d["kijun"])
        if cfg.confirm_chikou_above:
            confs.append(d["Close"].shift(-26) > d["Close"])
        if cfg.confirm_min_break_pct and cfg.confirm_min_break_pct > 0:
            confs.append(((d["Close"] / d["cloud_top"]) - 1.0) >= cfg.confirm_min_break_pct)
        if cfg.confirm_min_cloud_thickness_pct and cfg.confirm_min_cloud_thickness_pct > 0:
            confs.append(d["cloud_thickness"] >= cfg.confirm_min_cloud_thickness_pct)
        for c in confs:
            sig = sig & c

    if cfg.use_volume_confirm:
        vol_surge = d["Volume"] > (cfg.volume_surge_mult * d["avg_vol_ma"])
        sig = sig & vol_surge

    return sig

def indicator_exit_series(d: pd.DataFrame) -> pd.Series:
    return (d["Close"] < d["cloud_bot"]) | (d["Close"] < d["kijun"])

# =========================
# Benchmark / VOLAR
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_VOLAR(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, lb: int) -> Dict[str, float]:
    scores: Dict[str, float] = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lb:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty:
            scores[t] = 0.0; continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty:
                scores[t] = 0.0; continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lb:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lb)):
            scores[t] = 0.0; continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

# =========================
# Portfolio I/O
# =========================
REQ_POS_COLUMNS = ["ticker","entry_date","entry_price","shares","status"]

def load_portfolio(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        empty = pd.DataFrame(columns=REQ_POS_COLUMNS + ["exit_date","exit_price","stop_price","target_price","notes"])
        empty.to_csv(path, index=False)
        return empty
    df = pd.read_csv(path)
    df.columns = [c.strip().lower() for c in df.columns]
    for c in REQ_POS_COLUMNS:
        if c not in df.columns:
            df[c] = np.nan
    if "entry_date" in df.columns:
        df["entry_date"] = pd.to_datetime(df["entry_date"], errors="coerce")
    if "exit_date" in df.columns:
        df["exit_date"] = pd.to_datetime(df["exit_date"], errors="coerce")
    for numc in ["entry_price","shares","exit_price","stop_price","target_price"]:
        if numc in df.columns:
            df[numc] = pd.to_numeric(df[numc], errors="coerce")
    if "status" in df.columns:
        df["status"] = df["status"].astype(str).str.lower()
    return df

def save_portfolio(df: pd.DataFrame, path: str):
    cols = REQ_POS_COLUMNS + [c for c in df.columns if c not in REQ_POS_COLUMNS]
    df[cols].to_csv(path, index=False)

def append_entries_to_portfolio(port_df: pd.DataFrame, entry_plan: pd.DataFrame, as_of: pd.Timestamp, cfg: Config) -> pd.DataFrame:
    """Append new entries as OPEN with next-session entry_date, avoiding duplicates."""
    if entry_plan is None or entry_plan.empty:
        return port_df
    next_sess = next_weekday(as_of)

    # Existing open tickers (avoid duplicates)
    open_tickers = set(port_df.loc[port_df["status"] == "open", "ticker"].astype(str).str.upper())

    rows = []
    for _, r in entry_plan.iterrows():
        tkr = str(r["ticker"]).upper()
        if tkr in open_tickers:
            continue  # already open; skip

        est_entry = float(r["est_entry_price"])
        est_shares = int(r.get("est_shares", 0) or 0)
        stop_p = est_entry * (1 - cfg.stop_loss_pct)
        tgt_p  = est_entry * (1 + cfg.target_pct)

        rows.append({
            "ticker": tkr,
            "entry_date": next_sess.strftime("%Y-%m-%d"),
            "entry_price": est_entry,
            "shares": est_shares,
            "status": "open",
            "stop_price": stop_p,
            "target_price": tgt_p,
            "notes": "auto-appended from next_day_entries"
        })

    if not rows:
        return port_df

    add_df = pd.DataFrame(rows)
    add_df["entry_date"] = pd.to_datetime(add_df["entry_date"])
    combined = pd.concat([port_df, add_df], ignore_index=True)
    return combined

# =========================
# Telegram
# =========================
def send_telegram(msg: str, cfg: Config):
    if not cfg.telegram_enabled:
        return
    if not cfg.telegram_bot_token or not cfg.telegram_chat_id:
        log.warning("Telegram enabled but missing token/chat id.")
        return
    if requests is None:
        log.warning("requests not available; cannot send Telegram.")
        return
    try:
        url = f"https://api.telegram.org/bot{cfg.telegram_bot_token}/sendMessage"
        payload = {"chat_id": cfg.telegram_chat_id, "text": msg, "parse_mode": "HTML", "disable_web_page_preview": True}
        r = requests.post(url, json=payload, timeout=10)
        if r.status_code != 200:
            log.warning("Telegram send failed: %s | %s", r.status_code, r.text)
    except Exception as e:
        log.warning("Telegram exception: %s", e)

# =========================
# Build ENTRY / EXIT plans
# =========================
def build_entry_plan(cfg: Config, data_map: Dict[str, pd.DataFrame], as_of: pd.Timestamp) -> pd.DataFrame:
    enriched: Dict[str, pd.DataFrame] = {}
    for t, df in data_map.items():
        d = compute_ichimoku_base(df, cfg)
        if as_of not in d.index:
            d = d[d.index <= as_of]
            if d.empty: 
                continue
        enriched[t] = d

    candidates = []
    for t, d in enriched.items():
        sig = entry_signal_series(d, cfg)
        if as_of not in sig.index or not bool(sig.loc[as_of]):
            continue
        row = d.loc[as_of]
        c = float(row["Close"]); h52 = float(row["high_52w"])
        if not (h52 > 0 and c >= cfg.within_pct_of_52w_high * h52):
            continue
        if cfg.enable_basic_liquidity and (c < cfg.min_price_inr or row["avg_vol_20"] < cfg.min_avg_vol_20d):
            continue
        candidates.append({
            "ticker": t,
            "signal_date": as_of,
            "close": c,
            "cloud_top": float(row["cloud_top"]),
            "cloud_bot": float(row["cloud_bot"]),
            "span_a": float(row["span_a_base"]),
            "span_b": float(row["span_b_base"]),
            "kijun": float(row["kijun"]),
            "tenkan": float(row["tenkan"]),
            "high_52w": float(row["high_52w"]),
            "cloud_thickness": float(row["cloud_thickness"]),
            "dist_above_cloud": float(row["dist_above_cloud"]),
            "score": float(1000.0 * row["dist_above_cloud"]),
            "volume": float(row["Volume"]),
            "avg_vol_ma": float(row["avg_vol_ma"]),
        })

    if not candidates:
        return pd.DataFrame()

    cands_df = pd.DataFrame(candidates).reset_index(drop=True)

    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    volar_map = compute_VOLAR(as_of, cands_df["ticker"].tolist(), data_map, bench_df, cfg.volar_lookback)
    cands_df["volar"] = cands_df["ticker"].map(volar_map)
    cands_df["rank_score"] = cands_df["volar"].fillna(0.0) + 0.1*cands_df["score"].fillna(0.0)
    cands_df = cands_df.sort_values("rank_score", ascending=False).reset_index(drop=True)

    top = cands_df.head(cfg.top_k_daily).copy()
    if top.empty:
        return pd.DataFrame()

    deploy_cash = float(cfg.current_cash_inr) * float(cfg.deploy_cash_frac)
    per_pick_inr = deploy_cash / top.shape[0] if top.shape[0] > 0 else 0.0

    rows = []
    for i, r in top.iterrows():
        px = float(r["close"])
        shares = 0 if (px <= 0 or per_pick_inr < cfg.per_pick_min_inr) else int(math.floor(per_pick_inr / px))
        est_entry = px
        est_stop  = est_entry * (1 - cfg.stop_loss_pct)
        est_tgt   = est_entry * (1 + cfg.target_pct)
        reason_bits = ["Ichimoku breakout: Close↑CloudTop & SpanA>SpanB"]
        if cfg.use_ichimoku_confirms:
            sub=[]; 
            if cfg.confirm_tenkan_gt_kijun: sub.append("Tenkan>Kijun")
            if cfg.confirm_close_gt_kijun:  sub.append("Close>Kijun")
            if cfg.confirm_chikou_above:    sub.append("ChikouProxy↑")
            if cfg.confirm_min_break_pct and cfg.confirm_min_break_pct>0: sub.append(f"Break≥{cfg.confirm_min_break_pct:.2%}")
            if cfg.confirm_min_cloud_thickness_pct and cfg.confirm_min_cloud_thickness_pct>0: sub.append(f"Kumo≥{cfg.confirm_min_cloud_thickness_pct:.2%}")
            if sub: reason_bits.append("IchimokuConf[" + ",".join(sub) + "]")
        if cfg.use_volume_confirm:
            reason_bits.append(f"Vol>{cfg.volume_surge_mult:.1f}×MA{cfg.volume_ma_len}")

        rows.append({
            "rank": int(i+1),
            "ticker": r["ticker"],
            "signal_date": r["signal_date"].strftime("%Y-%m-%d"),
            "rank_score": float(r["rank_score"]),
            "volar": float(r["volar"]),
            "close": float(r["close"]),
            "cloud_top": float(r["cloud_top"]),
            "cloud_bot": float(r["cloud_bot"]),
            "kijun": float(r["kijun"]),
            "tenkan": float(r["tenkan"]),
            "dist_above_cloud": float(r["dist_above_cloud"]),
            "cloud_thickness": float(r["cloud_thickness"]),
            "within_52w_pct": float(r["close"]/r["high_52w"]) if r["high_52w"]>0 else np.nan,
            "est_entry_price": float(est_entry),
            "est_stop_price": float(est_stop),
            "est_target_price": float(est_tgt),
            "est_shares": int(shares),
            "est_alloc_inr": float(shares * est_entry),
            "reason": " ; ".join(reason_bits),
        })
    return pd.DataFrame(rows)

def build_exit_plan(cfg: Config, data_map: Dict[str, pd.DataFrame], as_of: pd.Timestamp, portfolio_df: pd.DataFrame) -> pd.DataFrame:
    if portfolio_df is None or portfolio_df.empty:
        return pd.DataFrame()
    open_pos = portfolio_df[portfolio_df["status"] == "open"].copy()
    if open_pos.empty:
        return pd.DataFrame()

    if "stop_price" not in open_pos.columns:
        open_pos["stop_price"] = np.nan
    if "target_price" not in open_pos.columns:
        open_pos["target_price"] = np.nan
    for idx, r in open_pos.iterrows():
        if pd.isna(r.get("stop_price")) and not pd.isna(r.get("entry_price")):
            open_pos.at[idx, "stop_price"] = float(r["entry_price"]) * (1 - cfg.stop_loss_pct)
        if pd.isna(r.get("target_price")) and not pd.isna(r.get("entry_price")):
            open_pos.at[idx, "target_price"] = float(r["entry_price"]) * (1 + cfg.target_pct)

    rows = []
    for _, pos in open_pos.iterrows():
        tkr = str(pos["ticker"]).upper().strip()
        df = data_map.get(tkr)
        if df is None or df.empty:
            continue
        d = compute_ichimoku_base(df, cfg)
        if as_of not in d.index:
            d = d[d.index <= as_of]
            if d.empty:
                continue
        row = d.loc[as_of]
        close = float(row["Close"])
        ind_exit = bool(indicator_exit_series(d).loc[as_of])

        eprice = float(pos.get("entry_price", np.nan))
        stop_p = float(pos.get("stop_price", np.nan))
        tgt_p  = float(pos.get("target_price", np.nan))
        stop_hit = (not np.isnan(stop_p)) and (close <= stop_p)
        tgt_hit  = (not np.isnan(tgt_p)) and (close >= tgt_p)

        reason_bits = []
        if ind_exit: reason_bits.append("IndicatorExit (Close<KumoBot or Close<Kijun)")
        if stop_hit: reason_bits.append("Close<=Stop (EOD)")
        if tgt_hit:  reason_bits.append("Close>=Target (EOD)")

        if reason_bits:
            rows.append({
                "ticker": tkr,
                "signal_date": as_of.strftime("%Y-%m-%d"),
                "close": close,
                "cloud_bot": float(row["cloud_bot"]),
                "kijun": float(row["kijun"]),
                "entry_date": pos.get("entry_date").strftime("%Y-%m-%d") if pd.notna(pos.get("entry_date")) else "",
                "entry_price": eprice,
                "stop_price": stop_p,
                "target_price": tgt_p,
                "advice": "EXIT ON OPEN",
                "reason": " ; ".join(reason_bits)
            })
    return pd.DataFrame(rows)

# =========================
# Main
# =========================
def main():
    # === Configure your universe here ===
    # CFG.static_symbols = [
    #     'RELIANCE.NS','TCS.NS','INFY.NS','HDFCBANK.NS','ICICIBANK.NS',
    #     'LT.NS','ITC.NS','SBIN.NS','BHARTIARTL.NS','HINDUNILVR.NS'
    # ]
    CFG.static_symbols_path = "nifty500.txt"
    CFG.use_ichimoku_confirms = True
    CFG.use_volume_confirm = False
    # Load universe data
    symbols = load_static_symbols(CFG.static_symbols, CFG.static_symbols_path)
    log.info("Universe: %d symbols", len(symbols))
    data_map = fetch_prices(symbols, CFG.start_date, CFG.end_date, CFG.cache_dir)
    data_map = {k:v for k,v in data_map.items() if v is not None and not v.empty}
    if not data_map:
        log.error("No data fetched.")
        return

    # As-of date (EOD)
    as_of = max(df.index[-1] for df in data_map.values())
    log.info("As-of date: %s", as_of.date())

    # ENTRY plan (new trades for next session)
    entry_plan = build_entry_plan(CFG, data_map, as_of)

    # Portfolio (auto-create if missing)
    portfolio_df = load_portfolio(CFG.portfolio_path)

    # Auto-append entries as OPEN (assume we take them)
    if CFG.auto_append_entries and entry_plan is not None and not entry_plan.empty:
        before_n = len(portfolio_df)
        portfolio_df = append_entries_to_portfolio(portfolio_df, entry_plan, as_of, CFG)
        after_n = len(portfolio_df)
        if after_n > before_n:
            save_portfolio(portfolio_df, CFG.portfolio_path)
            log.info("Auto-appended %d entries to portfolio.", after_n - before_n)

    # EXIT plan for current open trades
    exit_plan = build_exit_plan(CFG, data_map, as_of, portfolio_df)

    # Write artifacts
    out_dir = out_dir_for_today(CFG.out_root)

    # Entries
    entries_csv = os.path.join(out_dir, "next_day_entries.csv")
    entries_json = os.path.join(out_dir, "next_day_entries.json")
    if entry_plan is None or entry_plan.empty:
        with open(entries_json, "w") as f:
            json.dump({"as_of_date": str(as_of.date()), "entries": []}, f, indent=2)
        pd.DataFrame(columns=["rank","ticker","signal_date","est_entry_price","est_stop_price","est_target_price","est_shares"]).to_csv(entries_csv, index=False)
        log.info("No new entries.")
    else:
        entry_plan.to_csv(entries_csv, index=False)
        payload = {
            "as_of_date": str(as_of.date()),
            "config": {
                "use_ichimoku_confirms": CFG.use_ichimoku_confirms,
                "use_volume_confirm": CFG.use_volume_confirm,
                "top_k_daily": CFG.top_k_daily,
                "within_pct_of_52w_high": CFG.within_pct_of_52w_high,
                "deploy_cash_frac": CFG.deploy_cash_frac,
                "stop_loss_pct": CFG.stop_loss_pct,
                "target_pct": CFG.target_pct,
                "auto_append_entries": CFG.auto_append_entries,
            },
            "entries": entry_plan.to_dict(orient="records"),
        }
        with open(entries_json, "w") as f:
            json.dump(payload, f, indent=2)
        if CFG.telegram_enabled and CFG.alert_on_entries:
            lines = [f"📈 <b>Next-Day Entries ({as_of.date()})</b>"]
            for _, r in entry_plan.iterrows():
                lines.append(f"{int(r['rank']):>2}. <b>{r['ticker']}</b> ~{r['est_entry_price']:.2f} | SL {r['est_stop_price']:.2f} | TP {r['est_target_price']:.2f}")
            send_telegram("\n".join(lines), CFG)

    # Exits
    exits_csv = os.path.join(out_dir, "next_day_exits.csv")
    exits_json = os.path.join(out_dir, "next_day_exits.json")
    if exit_plan is None or exit_plan.empty:
        with open(exits_json, "w") as f:
            json.dump({"as_of_date": str(as_of.date()), "exits": []}, f, indent=2)
        pd.DataFrame(columns=["ticker","signal_date","advice","reason"]).to_csv(exits_csv, index=False)
        log.info("No exits triggered.")
    else:
        exit_plan.to_csv(exits_csv, index=False)
        payload = {
            "as_of_date": str(as_of.date()),
            "exits": exit_plan.to_dict(orient="records"),
        }
        with open(exits_json, "w") as f:
            json.dump(payload, f, indent=2)
        if CFG.telegram_enabled and CFG.alert_on_exits:
            lines = [f"📤 <b>Next-Day Exits ({as_of.date()})</b>"]
            for _, r in exit_plan.iterrows():
                lines.append(f"• <b>{r['ticker']}</b> EXIT ON OPEN | close {r['close']:.2f} | SL {r['stop_price']:.2f} | TP {r['target_price']:.2f} | {r['reason']}")
            send_telegram("\n".join(lines), CFG)

    log.info("Plans written to: %s", out_dir)
    log.info("Entries CSV: %s", entries_csv)
    log.info("Exits   CSV: %s", exits_csv)
    log.info("Portfolio file: %s", CFG.portfolio_path)

if __name__ == "__main__":
    main()


2025-11-06 23:45:04 | INFO | Universe: 500 symbols
2025-11-06 23:47:50 | INFO | As-of date: 2025-11-04
2025-11-06 23:47:51 | INFO | No new entries.
2025-11-06 23:47:51 | INFO | No exits triggered.
2025-11-06 23:47:51 | INFO | Plans written to: outputs/2025-11-06
2025-11-06 23:47:51 | INFO | Entries CSV: outputs/2025-11-06/next_day_entries.csv
2025-11-06 23:47:51 | INFO | Exits   CSV: outputs/2025-11-06/next_day_exits.csv
2025-11-06 23:47:51 | INFO | Portfolio file: portfolio_positions.csv
